# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'api_keys'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read output csv from part 1
city_csv = "../../output_data/cities.csv"

city_df = pd.read_csv(city_csv)

city_df

,Unnamed: 0,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Casablanca,33.5928,-7.6192,61.02,63,21,11.01,MA,1650406879
1,1,Olovyannaya,50.9500,115.5667,39.40,46,100,16.55,RU,1650406879
2,2,Lavrentiya,65.5833,-171.0000,29.07,74,3,6.15,RU,1650406880
3,3,Ubinskoye,55.3068,79.6802,36.48,63,100,6.80,RU,1650406880
4,4,New Norfolk,-42.7826,147.0587,56.70,76,97,3.11,AU,1650406881
...,...,...,...,...,...,...,...,...,...,...
498,498,Chekhov,55.1431,37.4711,44.85,95,89,10.96,RU,1650407635
499,499,Peterhead,57.5058,-1.7981,43.66,78,79,7.27,GB,1650407635
500,500,Sinaloa,25.0000,-107.5000,90.10,33,0,5.99,MX,1650407385
501,501,Ulundi,-28.3352,31.4162,54.63,95,1,3.65,ZA,1650407635


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
#Configure gmaps
gmaps.configure(api_key=g_key)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
